In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime as dt
import traceback
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# Filter subtitle

In [646]:
subtitles_to_remove = ['VR',
                       "WWE",
                       "Инсульт",
                       "Погладь",
                       "Fishing",
                       "Football",
                       "NBA",
                       "Golf",
                       "Let's Sing",
                       "MLB",
                       "My Universe",
                       "ACA NEOGEO",
                       "LEGO",
                       "Outbreak",
                       "The Escapists",
                       "Madden",
                       "NHL",
                       "Chess",
                       "Arcade Archives",
                       "FINAL FANTASY",
                       "Syberia",
                       "Assassin's Creed",
                       "Tennis",
                       "MFC",
                       "Disney",
                       "Pool",
                       "Tycoon",
                       "Погладить",
                       "STAR WARS",
                       "Call of Duty",
                       "Snooker",
                       "Tom Clancy's",
                       "Star Wars",
                       "Soccer",
                       "Baseball",
                       "Mega Man",
                       "NARUTO",
                       "Borderlands",
                       "Cricket",
                       "Саундтрек",
                       "Warhammer",
                       "South Park",
                       "Solitaire",
                       "EA SPORTS",
                       "Skyrim",
                       "Skate",
                       "JIGSAW",
                       "Poker",
                       
                      ]

In [647]:
title_substring = subtitles_to_remove[-1]

filename = "full_list_v2.csv"
df = pd.read_csv(filename)

for subtitle in subtitles_to_remove:
    df.loc[df[df['title'].str.contains(title_substring)].index, 'status'] = -1

df = df.sort_values(by=['status', 'title'], ascending=[False, True]).reset_index(drop=True)
df.to_csv(filename, index=False)


In [648]:
title_substring = subtitles_to_remove[-1]
df[df['title'].str.contains(title_substring)]

,title,platform,psd_URL,icon_URL,price,status
9374,Poker Club PS4 and PS5,PS5 / PS4,https://psdeals.net/ua-store/game/2128633/poke...,https://store.playstation.com/store/api/chihir...,649.00₴,-1
9375,Poker Club: Gold Edition PS4 and PS5,PS4,https://psdeals.net/ua-store/game/2690345/poke...,https://store.playstation.com/store/api/chihir...,719.00₴,-1
9376,Poker Mega Pack,PS4,https://psdeals.net/ua-store/game/1483520/poke...,https://store.playstation.com/store/api/chihir...,"1,499.00₴",-1
9377,Poker Pretty Girls Battle: Fantasy World Edition,PS5,https://psdeals.net/ua-store/game/2253092/poke...,https://store.playstation.com/store/api/chihir...,194.00₴,-1
9378,Poker Pretty Girls Battle: Fantasy World Edition,PS4,https://psdeals.net/ua-store/game/2198001/poke...,https://store.playstation.com/store/api/chihir...,194.00₴,-1
9379,Poker Pretty Girls Battle: Texas Hold'em PS4 a...,PS5 / PS4,https://psdeals.net/ua-store/game/2265553/poke...,https://store.playstation.com/store/api/chihir...,194.00₴,-1
9380,Poker Starter Pack,PS4,https://psdeals.net/ua-store/game/1427349/poke...,https://store.playstation.com/store/api/chihir...,789.00₴,-1
9538,Pure Hold'em World Poker Championship,PS4,https://psdeals.net/ua-store/game/7604/pure-ho...,https://store.playstation.com/store/api/chihir...,649.00₴,-1
12584,Video Poker,PS4,https://psdeals.net/ua-store/game/2667782/vide...,https://store.playstation.com/store/api/chihir...,99.00₴,-1


# Delete publisher

In [4]:
def get_pub(publisher):
    
    time = dt.now()
    
    page_num = 1
    df_pub = pd.DataFrame(columns=['title', 'platform', "psd_URL", "icon_URL", "price"])

    prev_page = None

    while True:
        try:
            URL = f"https://psdeals.net/ua-store/all-games/{page_num}?publisher={publisher}"
            #URL = f"https://psdeals.net/ua-store/all-games{page_num}?contentType%5B%5D=games&contentType%5B%5D=bundles&feature%5B%5D=vrGames"

            headers = requests.utils.default_headers()
            page = requests.get(URL, headers=headers)

            soup = BeautifulSoup(page.content, "html.parser")

            grid_items = soup.find_all('div', class_='game-collection-item col-md-2 col-sm-4 col-xs-6')

            if grid_items == prev_page:
                break

            for item in grid_items:

                title = item.find('span', class_='game-collection-item-details-title').text
                platform = item.find('span', class_='game-collection-item-top-platform').text
                image = item.find('img', class_='game-collection-item-image lazy').get('data-src')
                psd_link = "https://psdeals.net" + item.find('a', class_='game-collection-item-link').get('href')

                try:
                    price = item.find('span', class_='game-collection-item-price').text
                except:
                    price = np.NaN

                new_row = {"title": title, 
                           "platform": platform,
                           "price": price,
                           "psd_URL": psd_link,
                           "icon_URL": image}


                df_pub = df_pub.append(new_row, ignore_index=True)

            print(f"page {page_num}  time {dt.now() - time}")

            page_num += 1

            prev_page = grid_items

        except Exception as e:
            print(e)
            print(traceback.format_exc())
            break
            
    return df_pub

In [652]:
bad_publishers_list = ["Zakym+s.+r.+o.",
                       "ThiGames", 
                       "Kimulator%27s+Films+inc.", 
                       "SILESIA+GAMES+SP.+Z+O.O.",
                       "eastasiasoft",
                       "e-llusiontertainment",
                       "Webnetic+s.+r.+o.",
                       "Klovako+s.+r.+o.",
                       "THE+QUANTUM+ASTROPHYSICISTS+GUILD",
                       "SMOBILE+INC",
                       "Gametry+sp.+z+o.+o.",
                       "NOSTRA+GAMES+LTD",
                       "Benoit+Varasse+trading+as+Pix+Arts",
                       "Wild+River+Games",
                       "Acyntha",
                       "SOURCE+BYTE+SPÓŁKA+Z+OGRANICZONĄ+ODPOWIEDZIALNOŚCIĄ",
                       "Toplitz+Productions",
                       "Breakthrough+Gaming+LLC",
                       "YASH+FUTURE+TECH+SOLUTIONS+PVT+LTD",
                       "JOINDOTS+GMBH",
                       "4HIT",
                       "OCEAN+MEDIA+LLC",
                       "QUBYTE+INTERACTIVE",
                       "QUBICGAMES+SP.+Z+O.O.",
                       "Legacy+Interactive%2C+Inc.",
                       "Interactive+Dreams+Limited",
                       "ERIK+GAMES+YAZILIM+OYUN+TEKNOLOJILERI+TICARET+ANONIM+SIRKETI",
                       "Gameloft+SE",
                       "COATSINK+SOFTWARE+LTD",
                       "Thunderful+Games+Ltd",
                       "THE+VOICES+GAMES+LTD",
                       "PLAYSTIGE+INTERACTIVE+INFORMATION+AND+TRADE+LIMITED+LIABILITY+COMPANY",
                       "Aristo+Studio+s.r.o.",
                       "4HIT+LTD",
                       "KAIROSOFT+CO.%2CLTD",
                       "CFK+Co.%2C+Ltd.",
                       "ERDEM+SEN",
                       "KLABATER",
                       "ARCADE+DISTILLERY",
                       "OUTRIGHT+GAMES+LLC",
                       "KODOBUR+YAZILIM+BILISIM+TASARIM+ARG",
                       "Volkov+Konstantin+Sergeevich",
                       "TESEO+Y+EL+MINOTAURO%2C+S.L.U.",
                       "WEAKFISH+STUDIO+PUBLISHING",
                       "EVGENIY+KOLPAKOV",
                       "Gamuzumi",
                       "FOREVER+ENTERTAINMENT+S.+A.",
                       "Bigben+Interactive",
                       "MARKT%2BTECHNIK+VERLAG+GMBH",
                       "Scout+Game+Studios+Sociedad+Anonima",
                       "Xitilon",
                       "Baltoro+Games+Sp.z+o.o.",
                       "Armin+Unold",
                       "MOVING+PLAYER+SAS",
                       "Mike+Toth+trading+as+Toth+Games",
                       "FUNBOX+MEDIA+LIMITED",
                       "ARMOR+GAMES+INC",
                       "DIGERATI+DISTRIBUTION",
                       "CHUBBY+PIXEL",
                       "S.R.L.+%22RANDOMSPIN-GAMES%22",
                       "ARTIFEX+MUNDI+S.A.",
                       "Xeneder+Team",
                       "Revulo+Games+sp.+z+o.+o.",
                       "Skermunkel+Studios+%28Pty.%29+Ltd.",
                       "KEMCO",
                       "Maksym+Vysochanskyy+trading+as+IndieGames3000",
                       "Lightwood+Games",
                       "Colossus+Game+Studio+LTDA.",
                       "CATNESS+GAME+STUDIOS+SL",
                       "ARTIFEX+MUNDI+SP.+Z+O.O.",
                       "THOMAS+CLAUS+UND+FRANK+MENZEL",
                       "Set+Sail+Games+LLC",
                       "RATALAIKA+GAMES+SL",
                       "2Awesome+Studio++SL",
                       "PARADOX+INTERACTIVE+AB",
                       "Nakana+sp.+z+o.o.",
                       "GAMEMILL+ENTERTAINMENT",
                       "MARMALADE+GAME+STUDIO+LIMITED",
                       "Microids",
                       "BIG+WAY+GAMES",
                       "Dolores+Entertainment+S.L.",
                       "SABEC+LIMITED",
                       "Crunching+Koalas+Sp.+z+o.o.+Sp.+K.",
                       "Phoenix+Reborn+Games",
                       "Samustai+LTD",
                       "UNFINISHED+PIXEL%2C+SL",
                       "Aksys+Games+Localization+Inc.",
                       "Draknek+Limited",
                       "JANDUSOFT+S.L.",
                       "SUPER+POWERUP+GAMES%2C+SOCIEDAD+LIMITADA",
                       "Kanuni+games+oyun+YAZILIM+VE+Pazarlama+Ticaret+limited+Sirketi",
                       "AKUPARA+GAMES",
                       "Whitethorn+Games%2C+Inc.",
                       "ASSEMBLE+ENTERTAINMENT+GMBH",
                       "OÜ+Enningture+Game+Temple",
                       "Oiven+Games",
                       "Rainy+Frog+Co.+Ltd",
                       "Wales+Interactive+Limited",
                       "CHORUS+WORLDWIDE+GAMES",
                       "QUBICGAMES+S.A.",
                       "Curve+Digital",
                       "InfiniteZone+OÜ",
                       "TERNOX",
                       "UKIYO+PUBLISHING+LIMITED",
                       "Limited+Run+Games%2C+Inc.",
                       "EpiXR+Games+UG",
                       "Game+Achievements+Ltd",
                       "FLASHBULB+APS",
                       "FANTASTICO+STUDIO+SRL",
                       "Nellyvision+Ltd",
                       "D3PUBLISHER",
                       "Marvelous+Europe+Limited",
                       "indienova",
                       "VRKiwi",
                       "ULTIMATE+GAMES",
                       "Sometimes+You",
                       "WAYFORWARD+TECHNOLOGIES",
                       "Meridiem+Games+S.L",
                       "Focus+Entertainment&sort=az",
                       "DAEDALIC+ENTERTAINMENT",
                       "ASTRAGON+ENTERTAINMENT+GMBH",
                       "BANDAI+NAMCO+ENTERTAINMENT+EUROPE",
                       "Maximum+Entertainment",
                       "Warlock+Arts",
                       "CHIBIG%2C+S.L.U.",
                       "Afil+Games",
                       "y-zo+studio",
                       "HANDY+GAMES+GMBH",
                       "FusionPlay+GmbH",
                       "GOGAME+CONSOLE+PUBLISHER+LTD",
                       "SANUK+GAMES+SARL",
                       "TINYBUILD+LLC",
                       "MY.GAMES+B.V.",
                       "Puppet+Combo",
                       "CITY-CONNECTION+CO.%2C+LTD.",
                       "Expansive+Worlds+AB",
                       "GameToTop+Corporation+LTD",
                       "PQube+Ltd",
                       "INTRAGAMES+CO.%2C+LTD.",
                       "TOP+HAT+STUDIOS%2C+INC.",
                       "Indie+Remix+Studios+LLC",
                       "Koei+Tecmo+Europe+Ltd",
                       "HEADUP+GAMES+GMBH+CO.+KG",
                       "LOG+GAMES+LTD",
                       "LUCIANO+IURINO+T%2FA+TROGLOBYTES+GAMES",
                       "Progressive+Live+Studio+s.c.",
                       "Motorsport+Games",
                       "Raw+Fury+AB",
                       "ZEROUNO+GAMES",
                       "ESDigital+Games+LTD",
                       "VICTORY+ROAD",
                       "Dead+Drop+Studios+LLC",
                       "Untold+Tales+S.A.",
                       "Slitherine",
                       "PLAYISM",
                       "SAMURAI+PUNK+PTY+LTD",
                       "Zazenfly+Development+DOO",
                       "Ziggurat+Interactive%2C+Inc.",
                       "NUMSKULL+GAMES+LTD",
                       "Feardemic",
                       "indie.io",
                       "Ellada+Games",
                       "SCREENWAVE+MEDIA",
                       "CHETRUSCA+SOFTWORKS+S.R.L",
                       "AEROSOFT+GMBH",
                       "www.handy-games.com+GmbH",
                       "Kistler+Studios",
                       "SOEDESCO+PUBLISHING",
                       "Frozen+Way+S.A.",
                       "RED+ART+GAMES",
                       "KALYPSO+MEDIA",
                       "PLAYWAY+S.A",
                       "Vaka%2C+Inc.",
                       "Reef+Entertainment",
                       "NIS+America",
                       "DANGEN+ENTERTAINMENT",
                       "Disney+Interactive",
                       "ZEN+STUDIOS+LTD",
                       "Green+Man+Gaming+Publishing",
                       "SPIKE+CHUNSOFT%2C+INC.",
                       "DRAGEUS+GAMES+S.A.",
                       "Yogscast+Games+Limited",
                       "RAW+FURY",
                       "FRONTIER+DEVELOPMENTS+PLC",
                       "SURPRISE+ATTACK",
                       "HEADUP+GAMES",
                       "404+GAMES+LTD",
                       "COOL+DEVS+S.R.L",
                       "DANIEL+WENGENROTH",
                       "Gaijin+Distribution+KFT",
                       "ASI+GAMES+TECHNOLOGIES+LTD",
                       "THUNDERFUL+PUBLISHING",
                       "GRIMTALIN+SRL",
                       "Saber+Interactive%2C+Inc",
                       "SKYBOUND+INTERACTIVE+LLC",
                       "LAST+CHANCE+MEDIA",
                       "WIRED+PRODUCTIONS+LIMITED",
                       "SOEDESCO+PUBLISHING+B.V.",
                       "ATARI+EUROPE+S.A.S.U.",
                       "NACON+SA",
                       "FMV+Interactive+Studio+LLC",
                       "JACKBOX+GAMES%2C+INC.",
                       "PLAYTONIC+LTD",
                       "Mob+Entertainment%2C+Inc.",
                       "Midnight+Works+SRL",
                       "BLOWFISH+STUDIOS+PTY+LTD",
                       "UFO+Interactive+Games",
                       "VISION+GAMES+PUBLISHING",
                       "OutOfTheBit+Ltd",
                       "GRAFFITI+GAMES",
                       "VERTIGO+GAMES",
                       "QUBYTE+GAME+STUDIO%2C+LLC",
                       "Frozenbyte+LTD",
                       "MONSTER+COUCH+SP.+Z+O.O.",
                       "ARTAX+GAMES+SL",
                       "Alexander+Carparelli",
                       "AUTO+SLAVIC+d.o.o.",
                       "BeXide",
                       "Juan+Carlos+Moya+Toledo",
                       
                    ]

In [653]:
publisher = bad_publishers_list[-1]

df_pub = get_pub(publisher)        
        
df = pd.read_csv("full_list_v2.csv")
        
df.loc[df[df['title'].isin(df_pub.title)].index, 'status'] = -1
df = df.sort_values(by=['status', 'title'], ascending=[False, True]).reset_index(drop=True)
df.to_csv("full_list_v2.csv", index=False)

page 1  time 0:00:00.845764


In [654]:
df_pub

,title,platform,psd_URL,icon_URL,price
0,Welcome to Empyreum,PS5,https://psdeals.net/ua-store/game/2841218/welc...,https://store.playstation.com/store/api/chihir...,649.00₴
1,Spellcats Auto Card Tactics,PS4,https://psdeals.net/ua-store/game/2776247/spel...,https://store.playstation.com/store/api/chihir...,329.00₴
2,Spellcats Auto Card Tactics,PS5,https://psdeals.net/ua-store/game/2776246/spel...,https://store.playstation.com/store/api/chihir...,459.00₴
3,The Murder of Sherlock Holmes,PS5,https://psdeals.net/ua-store/game/2772167/the-...,https://store.playstation.com/store/api/chihir...,459.00₴
4,The Murder of Sherlock Holmes,PS4,https://psdeals.net/ua-store/game/2772166/the-...,https://store.playstation.com/store/api/chihir...,329.00₴


# Sort

In [598]:
df = pd.read_csv("full_list_v2.csv")
df = df.sort_values(by=['status', 'title'], ascending=[False, True]).reset_index(drop=True)
df.to_csv("full_list_v2.csv", index=False)

In [135]:
df.head(10)

,title,platform,psd_URL,icon_URL,price,status
0,#BLUD,PS4,https://psdeals.net/ua-store/game/2793941/blud,https://store.playstation.com/store/api/chihir...,819.00₴,1
1,ASTRO BOT,PS5,https://psdeals.net/ua-store/game/2787643/astr...,https://store.playstation.com/store/api/chihir...,"1,999.00₴",1
2,Ctrl Alt Ego,PS5,https://psdeals.net/ua-store/game/2843851/ctrl...,https://store.playstation.com/store/api/chihir...,819.00₴,1
3,DEATH STRANDING DIRECTOR’S CUT,PS5,https://psdeals.net/ua-store/game/2226925/deat...,https://store.playstation.com/store/api/chihir...,"1,399.00₴",1
4,Dishonored 2,PS4,https://psdeals.net/ua-store/game/885036/disho...,https://store.playstation.com/store/api/chihir...,649.00₴,1
5,Dishonored® Definitive Edition,PS4,https://psdeals.net/ua-store/game/7570/dishono...,https://store.playstation.com/store/api/chihir...,649.00₴,1
6,Dishonored®: Death of the Outsider™,PS4,https://psdeals.net/ua-store/game/1331208/dish...,https://store.playstation.com/store/api/chihir...,949.00₴,1
7,Disjunction,PS4,https://psdeals.net/ua-store/game/2153028/disj...,https://store.playstation.com/store/api/chihir...,519.00₴,1
8,Ghost of Tsushima DIRECTOR’S CUT,PS5 / PS4,https://psdeals.net/ua-store/game/2227365/ghos...,https://store.playstation.com/store/api/chihir...,"2,299.00₴",1
9,Ghostrunner,PS5 / PS4,https://psdeals.net/ua-store/game/2271937/ghos...,https://store.playstation.com/store/api/chihir...,949.00₴,1


# Delete duplicates

In [56]:
df = pd.read_csv("full_list_v2.csv")
df.drop_duplicates(subset=['title', 'platform', 'psd_URL'], keep=False, inplace=True)
df.to_csv("full_list_v2.csv", index=False)

In [57]:
df.head()

,title,platform,psd_URL,icon_URL,price,status
0,ASTRO BOT,PS5,https://psdeals.net/ua-store/game/2787643/astr...,https://store.playstation.com/store/api/chihir...,"1,999.00₴",1
1,DEATH STRANDING DIRECTOR’S CUT,PS5,https://psdeals.net/ua-store/game/2226925/deat...,https://store.playstation.com/store/api/chihir...,"1,399.00₴",1
2,Ghost of Tsushima DIRECTOR’S CUT,PS5 / PS4,https://psdeals.net/ua-store/game/2227365/ghos...,https://store.playstation.com/store/api/chihir...,"2,299.00₴",1
3,God of War Рагнарёк,PS5 / PS4,https://psdeals.net/ua-store/game/2421397/god-...,https://store.playstation.com/store/api/chihir...,"2,299.00₴",1
4,God of War™,PS4,https://psdeals.net/ua-store/game/1901934/god-...,https://store.playstation.com/store/api/chihir...,649.00₴,1


# Liked games

In [ ]:
liked_lst = ['Alone in the Dark',
             "Wreckfest PlayStation®5 Version",
             "Cloudpunk",
             "The Exit 8",
             "Terminator: Resistance",
             "Deliver Us The Moon",
             "RoboCop: Rogue City",
             "Infliction: Extended Cut",
            
            ]